In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from unidecode import unidecode
import time



In [2]:
# Lee el archivo CSV
df_provinces = pd.read_csv('../data/lista_provincias.csv')

# Provincias sin Buenos Aires, Capital y Córdoba
# Eliminar las filas correspondientes a las provincias especificadas
provincias_a_eliminar = ["buenos-aires", "buenos-aires-(caba)", "cordoba"]
df_provinces = df_provinces[~df_provinces["Provincia"].isin(provincias_a_eliminar)]

In [ ]:
base_url = "https://codigo-postal.co/argentina/"

# Para cada provincia en el DataFrame
for index, row in df_provinces.iterrows():
    # Carga el archivo CSV de localidades para la provincia actual
    df_localities = pd.read_csv(f'../data/provincias/{row["Provincia"]}_localidad.csv')

    # Crea una lista vacía para almacenar los datos extraídos
    data = []

    # Para cada localidad en el DataFrame
    for i, r in df_localities.iterrows():
        # Crea la URL para la localidad
        locality_url = base_url + row['Provincia'] + '/' + r['Localidad']
       # Para cada provincia en el DataFrame
for index, row in df_provinces.iterrows():
    provincia = row["Provincia"]
    
    # Evita que se ejecute el código para las provincias especificadas
    if provincia not in ["buenos-aires", "buenos-aires(caba)", "cordoba"]:
        # Carga el archivo CSV de localidades para la provincia actual
        df_localities = pd.read_csv(f'../data/provincias/{provincia}_localidad.csv')

        # Crea una lista vacía para almacenar los datos extraídos
        data = []

        # Para cada localidad en el DataFrame
        for i, r in df_localities.iterrows():
            # Crea la URL para la localidad
            locality_url = base_url + provincia + '/' + r['Localidad']
           
            # Solicita la página
            response = requests.get(locality_url)
            
            # Analiza el HTML de la página
            soup = BeautifulSoup(response.text, 'html.parser')

            # Encuentra el CPA en la página
            cpa_link = soup.find('tbody').find('a') if soup.find('tbody') else None
            if cpa_link:
                cpa = cpa_link.text.strip()
                
                # Añade la localidad y el CPA a la lista
                data.append({"Localidad": r['Localidad'], "CPA": cpa})

            # Pausa para evitar sobrecargar el servidor
            time.sleep(.5)
        
        # Convierte la lista en un DataFrame
        df_cpa = pd.DataFrame(data)
        
        # Guarda el DataFrame en un archivo CSV
        df_cpa.to_csv(f'{provincia}_cpa.csv', index=False)

        # Solicita la página
        response = requests.get(locality_url)
        
        # Analiza el HTML de la página
        soup = BeautifulSoup(response.text, 'html.parser')

        # Encuentra el CPA en la página
        cpa_link = soup.find('tbody').find('a') if soup.find('tbody') else None
        if cpa_link:
            cpa = cpa_link.text.strip()
            
            # Añade la localidad y el CPA a la lista
            data.append({"Localidad": r['Localidad'], "CPA": cpa})

        # Pausa para evitar sobrecargar el servidor
        time.sleep(.5)
    
    # Convierte la lista en un DataFrame
    df_cpa = pd.DataFrame(data)
    
    # Guarda el DataFrame en un archivo CSV
    df_cpa.to_csv(f'../target/provinces/{row["Provincia"]}_cpa.csv', index=False)
